In [22]:
# llm 호출 및 langchain의 프롬프트를 사용하기 위함
%pip install langchain-aws==0.2.25

# .env를 사용하기 위함
%pip install python-dotenv==1.1.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_aws import ChatBedrock
from dotenv import load_dotenv
import os

# .env 내용 불러오기
load_dotenv(dotenv_path="../.env")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

llm = ChatBedrock(
    model_id="apac.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_kwargs=dict(temperature=0.1),
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name="ap-northeast-2"  # 서울 리전
)

In [18]:
response = llm.invoke("안녕하세요?")
print(response)

content='안녕하세요! 어떻게 도와드릴까요? 질문이나 도움이 필요한 것이 있으시면 말씀해 주세요.' additional_kwargs={'usage': {'prompt_tokens': 15, 'completion_tokens': 56, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 71}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'apac.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'apac.anthropic.claude-3-7-sonnet-20250219-v1:0'} response_metadata={'usage': {'prompt_tokens': 15, 'completion_tokens': 56, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 71}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'apac.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'apac.anthropic.claude-3-7-sonnet-20250219-v1:0'} id='run--e322c652-be83-41a9-909e-4a0252c05449-0' usage_metadata={'input_tokens': 15, 'output_tokens': 56, 'total_tokens': 71, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


In [3]:
# pdf파일을 로딩
%pip install -q pypdf==5.6.0
# langchain을 사용하기위함
%pip install -q langchain_community==0.3.25

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("doc/국민연금법.pdf")
docs = loader.load()
pages = loader.load()

print(len(pages))

38


In [22]:
%pip install -q langchain-text-splitters==0.3.8

Note: you may need to restart the kernel to use updated packages.


In [13]:
#text spliter
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list=loader.load_and_split(text_splitter=text_splitter)

In [14]:
# embedding
from langchain_aws import BedrockEmbeddings

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")

In [21]:
# chroma DB
%pip install -q langchain-chroma==0.2.4

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_chroma import Chroma

# 로컬 경로 지정
local_path = "vector"

database = Chroma.from_documents(
    documents=document_list, 
    embedding=bedrock_embeddings, 
    persist_directory=local_path
    )

In [16]:
query='국민연금의 수령 나이는 언제 인가요?'
retrieved_docs = database.similarity_search(query=query, k=3)
retrieved_docs

[Document(id='1660704a-a073-4a1c-9211-ae8cb533bedb', metadata={'page': 3, 'source': 'doc/국민연금법.pdf', 'page_label': '4', 'moddate': '2025-02-25T23:58:24+09:00', 'total_pages': 38, 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2025-02-25T23:58:24+09:00', 'creator': 'PyPDF'}, page_content='6. 사업장가입자 또는 지역가입자의 자격을 취득한 때\n7. 제6조 단서에 따른 국민연금 가입 대상 제외자에 해당하게 된 때\n \n제13조(임의계속가입자) ① 다음 각 호의 어느 하나에 해당하는 자는 제6조 본문에도 불구하고 65세가 될 때까지 보건\n복지부령으로 정하는 바에 따라 국민연금공단에 가입을 신청하면 임의계속가입자가 될 수 있다. 이 경우 가입 신청\n이 수리된 날에 그 자격을 취득한다. <개정 2008. 2. 29., 2010. 1. 18., 2011. 6. 7., 2011. 12. 31., 2015. 1. 28., 2016. 5.\n29.>\n1. 국민연금 가입자 또는 가입자였던 자로서 60세가 된 자. 다만, 다음 각 목의 어느 하나에 해당하는 자는 제외한다.\n가. 연금보험료를 납부한 사실이 없는 자\n나. 노령연금 수급권자로서 급여를 지급받고 있는 자\n다. 제77조제1항제1호에 해당하는 사유로 반환일시금을 지급받은 자'),
 Document(id='8abefcc8-6d1d-4e22-a006-d5764d269cd4', metadata={'creationdate': '2025-02-25T23:58:24+09:00', 'creator': 'PyPDF', 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'doc/국민연금법.pdf', 'page': 3, 'total_pages': 38, 'pag

In [9]:
from langchain.prompts import PromptTemplate

template = """"""

# PromptTemplate 정의
template = """[Identity]
- 당신은 대한민국 국민연금 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변하세요.
- 정보가 해당하는 법령도 포함해서 알려주세요.

[Context]
{docs}

question : {question}
"""

prompt = PromptTemplate(
    input_variables=["docs", "question"],
    template=template
)

question='국민연금의 수령 나이는 언제 인가요?'

ret_doc = database.similarity_search(query=question, k=3)

# 작성한 PromptTemplate에 문서와 질문을 삽입
formatted_prompt = prompt.format(docs=ret_doc, question=question)

# LLM에 프롬프트 전달 및 응답 받기
response = llm.invoke(formatted_prompt)
print(response.content)

국민연금의 수령 나이에 대해 답변드리겠습니다.

제공된 정보에 따르면, 국민연금 가입자는 일반적으로 60세가 되면 가입자 자격을 상실하게 됩니다. 이는 국민연금법 제12조에 명시되어 있습니다:

- 사업장가입자: 제12조 제1항 제4호에 따라 "60세가 된 때" 자격 상실
- 지역가입자: 제12조 제2항 제6호에 따라 "60세가 된 때" 자격 상실
- 임의가입자: 제12조 제3항 제4호에 따라 "60세가 된 때" 자격 상실

다만, 국민연금법 제13조에 따르면 임의계속가입자 제도가 있어, 다음 조건을 충족하는 경우 65세까지 계속해서 가입할 수 있습니다:
- 국민연금 가입자 또는 가입자였던 자로서 60세가 된 자
- 단, 다음에 해당하는 자는 제외:
  - 연금보험료를 납부한 사실이 없는 자
  - 노령연금 수급권자로서 급여를 지급받고 있는 자
  - 제77조제1항제1호에 해당하는 사유로 반환일시금을 지급받은 자

따라서 국민연금은 기본적으로 60세부터 수령할 수 있으며, 원하는 경우 65세까지 임의계속가입을 통해 가입 기간을 연장할 수 있습니다.

관련 법령: 국민연금법 제12조(가입자 자격의 상실 시기), 제13조(임의계속가입자)


In [17]:
# Memory
 
from langchain.memory import ConversationBufferMemory # 모든 대화내용 저장
# from langchain.memory import ConversationBufferWindowMemory # 가장 최근 k개 저장
from langchain.chains import ConversationalRetrievalChain
 
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# memory_w = ConversationBufferWindowMemory(memory_key="history", return_messages=True, k=5)
retriever = database.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)
 
qa("국민연금의 수령 나이는 언제 인가요?")

{'question': '국민연금의 수령 나이는 언제 인가요?',
 'chat_history': [HumanMessage(content='국민연금의 수령 나이는 언제 인가요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='제공된 정보에 따르면, 국민연금의 수령 나이에 대한 명확한 정보는 없습니다. 다만, 국민연금법에서는 가입자 자격과 관련하여 다음과 같은 내용이 있습니다:\n\n1. 사업장가입자와 지역가입자는 60세가 된 때에 자격을 상실합니다(제12조 제1항 제4호, 제2항 제6호).\n\n2. 임의계속가입자의 경우, 국민연금 가입자 또는 가입자였던 자로서 60세가 된 자는 65세가 될 때까지 임의계속가입자가 될 수 있습니다(제13조 제1항).\n\n이를 통해 일반적인 국민연금 가입 자격은 60세까지이며, 임의계속가입을 통해 65세까지 연장할 수 있음을 알 수 있습니다. 그러나 실제 연금 수령 시작 나이에 대한 구체적인 정보는 제공된 자료에 명시되어 있지 않습니다.', additional_kwargs={}, response_metadata={})],
 'answer': '제공된 정보에 따르면, 국민연금의 수령 나이에 대한 명확한 정보는 없습니다. 다만, 국민연금법에서는 가입자 자격과 관련하여 다음과 같은 내용이 있습니다:\n\n1. 사업장가입자와 지역가입자는 60세가 된 때에 자격을 상실합니다(제12조 제1항 제4호, 제2항 제6호).\n\n2. 임의계속가입자의 경우, 국민연금 가입자 또는 가입자였던 자로서 60세가 된 자는 65세가 될 때까지 임의계속가입자가 될 수 있습니다(제13조 제1항).\n\n이를 통해 일반적인 국민연금 가입 자격은 60세까지이며, 임의계속가입을 통해 65세까지 연장할 수 있음을 알 수 있습니다. 그러나 실제 연금 수령 시작 나이에 대한 구체적인 정보는 제공된 자료에 명시되어 있지 않습니다.'}

In [18]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# 질문을 위한 프롬프트 템플릿 생성
# AI가 대화의 맥락을 이해하고 적절한 답변을 할수 있도록함
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""
다음은 사람과 AI 어시스턴트의 친근한 대화입니다.

대화 기록: {chat_history}
인간: {question}

AI 어시스턴트: 이전 대화를 고려하여 질문에 답변해 드리겠습니다.
""")

# QA를 위한 프롬프트 템플릿 생성
QA_PROMPT = PromptTemplate.from_template("""
당신은 국민연금에 대해 전문적인 지식을 가진 상담사입니다.
주어진 컨텍스트를 기반으로 질문에 명확하고 정확하게 답변해 주세요.
정보가 해당하는 법령도 포함해서 알려주세요.
정보를 찾은 PDF페이지 번호도 알려주세요.

컨텍스트: {context}
질문: {question}

답변:
""")

# 메모리 설정
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
retriever = database.as_retriever()

# ConversationalRetrievalChain 생성 시 프롬프트 템플릿 적용
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs={'prompt': QA_PROMPT}
)

# 체인 실행
result = qa.invoke({
    "question": "국민연금의 수령 나이는 언제 인가요?"
})
print(result['answer'])

# 국민연금 수령 나이 안내

국민연금법에 따르면, 국민연금의 기본적인 수령 나이는 **60세**입니다. 이는 제12조(가입자 자격의 상실 시기)에서 확인할 수 있습니다.

구체적으로:
- 사업장가입자는 60세가 된 때 자격을 상실합니다(제12조 제1항 제4호)
- 지역가입자도 60세가 된 때 자격을 상실합니다(제12조 제2항 제6호)
- 임의가입자 역시 60세가 된 때 자격을 상실합니다(제12조 제3항 제4호)

다만, 제13조(임의계속가입자)에 따르면, 국민연금 가입자 또는 가입자였던 자로서 60세가 된 자는 65세가 될 때까지 임의계속가입자로 가입을 신청할 수 있습니다. 단, 연금보험료를 납부한 사실이 없는 자, 노령연금 수급권자로서 급여를 지급받고 있는 자, 반환일시금을 지급받은 자는 제외됩니다.

**관련 법령**: 국민연금법 제12조, 제13조
**PDF 페이지 번호**: 4


In [20]:
result = qa.invoke({
    "question": "70세 받고싶은데 가능한가요?"
})
# print(result)
print(result['answer'])

{'question': '70세 받고싶은데 가능한가요?', 'chat_history': [HumanMessage(content='국민연금의 수령 나이는 언제 인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='# 국민연금 수령 나이 안내\n\n국민연금법에 따르면, 국민연금의 기본적인 수령 나이는 **60세**입니다. 이는 제12조(가입자 자격의 상실 시기)에서 확인할 수 있습니다.\n\n구체적으로:\n- 사업장가입자는 60세가 된 때 자격을 상실합니다(제12조 제1항 제4호)\n- 지역가입자도 60세가 된 때 자격을 상실합니다(제12조 제2항 제6호)\n- 임의가입자 역시 60세가 된 때 자격을 상실합니다(제12조 제3항 제4호)\n\n다만, 제13조(임의계속가입자)에 따르면, 국민연금 가입자 또는 가입자였던 자로서 60세가 된 자는 65세가 될 때까지 임의계속가입자로 가입을 신청할 수 있습니다. 단, 연금보험료를 납부한 사실이 없는 자, 노령연금 수급권자로서 급여를 지급받고 있는 자, 반환일시금을 지급받은 자는 제외됩니다.\n\n**관련 법령**: 국민연금법 제12조, 제13조\n**PDF 페이지 번호**: 4', additional_kwargs={}, response_metadata={}), HumanMessage(content='70세 받고싶은데 가능해?', additional_kwargs={}, response_metadata={}), AIMessage(content='# 국민연금 수령 연기 가능 여부 안내\n\n안녕하세요, 국민연금 상담사입니다. 국민연금 수령 연기에 관한 질문에 답변드리겠습니다.\n\n## 국민연금 수령 연기 관련 정보\n\n국민연금법에 따르면, 노령연금 수급권자는 최대 65세까지만 연금 수령을 연기할 수 있습니다.\n\n- **법적 근거**: 국민연금법 제62조(지급의 연기에 따른 가산) 제1항\n- **내용**: "제61조에 따른 노령연금의 수급권자로서 6